In [1]:
import os
import json
import pymongo
root_dir = 'all-rnr-annotated-threads'
list_threads = os.listdir(root_dir)

### Connect ke MongoDB

In [2]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['dataset_graph']
collection = db[root_dir]

In [3]:
client.list_database_names()

['admin', 'config', 'dataset_graph', 'local']

In [4]:
db.list_collection_names()

['all-rnr-annotated-threads']

In [8]:
# Reset database
for c in db.list_collection_names(): db.drop_collection(c)

### Mendapatkan Dataset Tree

In [5]:
list_data = list()
for thread in list_threads:
    current_thread_dir = '{}/{}'.format(root_dir, thread, )
    data = dict()
    data['thread_name'] = thread
    data['non-rumours'] = list()
    data['rumours'] = list()
    
    # Untuk non rumours
    for id_tweet in os.listdir('{}/non-rumours'.format(current_thread_dir)):
        if not id_tweet.isnumeric(): continue
        try:
            dir_path = '{}/non-rumours/{}'.format(current_thread_dir, id_tweet)
            data_tweet = dict()
            data_tweet['id_tweet'] = id_tweet
            data_tweet['path'] = dir_path
            data_tweet['annotation'] = json.load(open('{}/annotation.json'.format(dir_path)))
            data_tweet['reactions'] = list()
            data_tweet['source_tweets'] = json.load(open('{}/source-tweets/{}.json'.format(dir_path, id_tweet)))
            data_tweet['structure'] = json.load(open('{}/structure.json'.format(dir_path)))
            data['non-rumours'].append(data_tweet)
        except NotADirectoryError:
            pass

    # Untuk rumours
    for id_tweet in os.listdir('{}/rumours'.format(current_thread_dir)):
        if not id_tweet.isnumeric(): continue
        try:
            dir_path = '{}/rumours/{}'.format(current_thread_dir, id_tweet)
            data_tweet = dict()
            data_tweet['id_tweet'] = id_tweet
            data_tweet['path'] = dir_path
            data_tweet['annotation'] = json.load(open('{}/annotation.json'.format(dir_path)))
            data_tweet['reactions'] = list()
            data_tweet['source_tweets'] = json.load(open('{}/source-tweets/{}.json'.format(dir_path, id_tweet)))
            data_tweet['structure'] = json.load(open('{}/structure.json'.format(dir_path)))
            data['rumours'].append(data_tweet)
        except NotADirectoryError:
            pass
    
    list_data.append(data)

In [ ]:
# disimpan sebagai file .json
json.dump(list_data, open('dataset_tree.json', 'w'))

In [11]:
# disimpan di MongoDB
if collection.find_one():  # sudah ada data
    print('update')
    for a in range(len(list_threads)):
        collection.find_one_and_replace(filter=dict(thread_name=list_threads[a]), replacement=list_data[a])
else:
    print('insert')
    collection.insert_many(list_data)

update


In [10]:
for c in collection.find():
    print(c['thread_name'])

charliehebdo-all-rnr-threads
ebola-essien-all-rnr-threads
ferguson-all-rnr-threads
germanwings-crash-all-rnr-threads
gurlitt-all-rnr-threads
ottawashooting-all-rnr-threads
prince-toronto-all-rnr-threads
putinmissing-all-rnr-threads
sydneysiege-all-rnr-threads


### Load data sesuai dengan kebutuhan

In [ ]:
raw_data_tree = json.load(open('dataset_tree.json'))
sample_thread = list_threads[0]
sample_list_tweet = list(raw_data_tree[sample_thread]['non-rumours'].keys())
(_, sample_data), (_, sample_path_data) = raw_data_tree[sample_thread]['non-rumours'][sample_list_tweet[0]].items()

In [ ]:
sample_data